In [ ]:
import csv
import ast
import time
import random
import numpy as np
import networkx as nx
import strawberryfields as sf
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from strawberryfields import ops
from itertools import combinations
from strawberryfields.apps import data, sample, subgraph, plot, qchem
from strawberryfields.apps.similarity import feature_vector_orbits_sampling
from strawberryfields.utils import random_interferometer
from collections import Counter
csv.field_size_limit(500 * 1024 * 1024)

Step1: Mapping

In [ ]:
# function that take a raw matrix and k (number of columns) and n (the nth number you want to compare) value in, return the best k columns
def find_best_k_combination(matrix, k, n):
    # get all possible combination
    num_columns = matrix.shape[1]
    column_combinations = combinations(range(num_columns), k)
        
#     column_combinations = get_column_combinations(matrix, k)
    
    strength = []
    Combination = []

    for combination in column_combinations:
        Combination.append(combination)
        selected_columns = matrix[:, combination]
        # calculate the row abs square sum
        row_sums = np.sum(np.abs(selected_columns) ** 2, axis=1)
        # make it decrease
        sorted_indices = np.argsort(row_sums)[::-1]
        sorted_row_sums = row_sums[sorted_indices]
        val = sorted_row_sums[n-1]
        strength.append(val)

    largest_value = np.max(strength)
    largest_index = np.argmax(strength)

    # find the best combination
    best_combination = Combination[largest_index]
    best_selected_columns = matrix[:, best_combination]

    return best_selected_columns

In [ ]:
# final function, recieve the original matrix and a list of len, also n (the nth number you want to compare), return the permuted matrix
# U = row_per_matrix*U'*col_per_matrix
def mapping(U, len_list, n):
    
    M, N = U.shape
    total_len = sum(len_list)
    assert N == total_len
    
    # copy U
    U_t = U.copy()
#     print(U)
    
    # new column permutation
    for i, length in enumerate(len_list):
        if i<len(len_list)-1:
            U_new = U_t[:,:length].copy()
#             print("i:", i)
            # calculate the row abs square sum
            row_sums = np.sum(np.abs(U_new) ** 2, axis=1)
            U_new_update = U_new.copy()
            # iterate remain column
            for j in range(np.sum(len_list[i+1:])):
#                 print("j:", j)
                for k in range(length):
#                     print("k:", k)
                    U_new_update_k = U_new_update[:,k].copy()
                    U_new_update[:,k] = U_t[:,length+j].copy()
                    # calculate the row abs square sum
                    row_sums_update = np.sum(np.abs(U_new_update) ** 2, axis=1)
                
                    if np.sort(row_sums_update)[::-1][n-1] > np.sort(row_sums)[::-1][n-1]:
#                     row_sums_update[n-1] > row_sums[n-1]:
                        U_new = U_new_update.copy()
                        U_t[:,length+j] = U_new_update_k.copy()
                        U_t[:,:length] = U_new.copy()
                        row_sums = row_sums_update
                    else:
                        U_new_update = U_new.copy()

            
            # add U_new to U_per
            if i==0:
                U_per = U_new.copy()
            elif i==len(len_list)-2:
                U_per = np.hstack((U_per, U_t)).copy()
            else:
                U_per = np.hstack((U_per, U_new)).copy()

            
            # drop U_new from U_t
            U_t = U_t[:,length:].copy()
        
                    

#     for i, length in enumerate(len_list):
#         U_new = find_best_k_combination(U_t, length, n)
        
#         # Find column indices in the larger matrix corresponding to the smaller matrix
#         selected_columns = []
#         for col in range(U_new.shape[1]):
#             for j in range(U_t.shape[1]):
#                 if np.array_equal(U_new[:, col], U_t[:, j]):
#                     selected_columns.append(j)
#                     break

#         # drop U_new from U_t
#         U_t = np.delete(U_t, selected_columns, axis=1)
        
#         # add U_new to U_per
#         if i==0:
#             U_per = U_new
#         else:
#             U_per = np.hstack((U_per, U_new))
      
      
    # find the permutation from U_per to U
    col_permutation = []
    for col in range(U_per.shape[1]):
        for i in range(U.shape[1]):
            if np.array_equal(U_per[:, col], U[:, i]):
                col_permutation.append(i)
                break
                
    # find the columns permutation matrix
    col_per_matrix = np.zeros((N, N))
    for i in range(N):
        col_per_matrix[col_permutation[i], i] = 1
        


    # calculate the sum of first len for U_per
    selected_columns = U_per[:, :len_list[0]] 
    # calculate the row abs square sum
    row_sums = np.sum(np.abs(selected_columns) ** 2, axis=1)
    
    # decide the row rotation
    sorted_indices = np.argsort(row_sums)
    U_per_new = U_per[sorted_indices]
    
    # Find row permutation
    row_permutation = []
    for row in range(U_per_new.shape[0]):
        for i in range(U_per.shape[0]):
            if np.array_equal(U_per_new[row, :], U_per[i, :]):
                row_permutation.append(i)
                break
                
    # find the rows permutation matrix
    row_per_matrix = np.zeros((M, M))
    for i in range(M):
        row_per_matrix[i, row_permutation[i]] = 1
        
    
    U_final = U_per_new
    
    # U_final = U_per, out put U_final and the permutation
    return U_final, col_per_matrix.T, row_per_matrix.T

Step2: Matrix Decomposition

In [ ]:
# Define the left eliminate equations
def left_equations(variables,parameters):
    theta, phi = variables
    a_r, a_i, b_r, b_i = parameters
    eq1 = a_r * np.cos(phi) * np.cos(theta) + a_i * np.sin(phi) * np.cos(theta) - b_r * np.sin(theta)
    eq2 = a_i * np.cos(phi) * np.cos(theta) - a_r * np.sin(phi) * np.cos(theta) - b_i * np.sin(theta)
    return [eq1, eq2]

# function of left elimination
def left_elimination(a,b):
    a_r = np.real(a)
    a_i = np.imag(a)
    b_r = np.real(b)
    b_i = np.imag(b)
    
    # Solve the system of equations
    initial_guess = [0, 0]  # Initial guess for the variables
    parameters = [a_r, a_i, b_r, b_i]
    raw_solution = fsolve(left_equations, initial_guess, args=(parameters,))
    
    
    if np.abs(a_r) < 1e-8 and np.abs(b_i) < 1e-8:
            raw_solution = [np.arctan(a_i/b_r), np.pi/2]
        
    if np.abs(a_i) < 1e-8 and np.abs(b_r) < 1e-8:
            raw_solution = [np.arctan(-a_r/b_i), np.pi/2]
            
            
    if np.abs(b_r) < 1e-8 and np.abs(b_i) < 1e-8:
            raw_solution = [np.pi/2, 0]
    
    
    normalized_solution = [angle % (2 * np.pi) for angle in raw_solution]
    solution = []
    for angle in normalized_solution:
        if angle < np.pi:
            solution.append(angle)
        else:
            solution.append(angle-2*np.pi)

    # Print the solution
#     print("Solution:", solution)
    return solution

In [ ]:
# Define the right eliminate equations
def right_equations(variables,parameters):
    theta, phi = variables
    a_r, a_i, b_r, b_i = parameters
    eq1 = a_r * np.cos(phi) * np.sin(theta) + a_i * np.sin(phi) * np.sin(theta) + b_r * np.cos(theta)
    eq2 = a_i * np.cos(phi) * np.sin(theta) - a_r * np.sin(phi) * np.sin(theta) + b_i * np.cos(theta)
    return [eq1, eq2]

# function of right elimination
def right_elimination(a,b):
    a_r = np.real(a)
    a_i = np.imag(a)
    b_r = np.real(b)
    b_i = np.imag(b)
    
    # Solve the system of equations
    initial_guess = [0, 0]  # Initial guess for the variables
    parameters = [a_r, a_i, b_r, b_i]
    raw_solution = fsolve(right_equations, initial_guess, args=(parameters,))
    
    if np.abs(a_r) < 1e-8 and np.abs(b_i) < 1e-8:
        raw_solution = [np.arctan(-b_r/a_i), np.pi/2]
        
    if np.abs(a_i) < 1e-8 and np.abs(b_r) < 1e-8:
        raw_solution = [np.arctan(b_i/a_r), np.pi/2]
        
    if np.abs(a_r) < 1e-8 and np.abs(a_i) < 1e-8:
        raw_solution = [np.pi/2, 0]
    
    normalized_solution = [angle % (2 * np.pi) for angle in raw_solution]
    solution = []
    for angle in normalized_solution:
        if angle < np.pi:
            solution.append(angle)
        else:
            solution.append(angle-2*np.pi)

    # Print the solution
#     print("Solution:", solution)
    return solution

In [ ]:
# give rotation matrix with idx_1 < idx_2
def rotation(theta, phi, n, idx_1, idx_2):
    I = np.eye(n, dtype=complex)
    I[idx_1,idx_1] = np.cos(theta)*(np.cos(phi)-np.sin(phi)*1j)
    I[idx_1,idx_2] = np.sin(theta)*(np.cos(phi)-np.sin(phi)*1j)
    I[idx_2,idx_1] = -np.sin(theta)
    I[idx_2,idx_2] = np.cos(theta)
    
    return I

In [ ]:
# matrix permutation
def permutation(U):
    
    rows, columns = U.shape
    assert(rows==columns)
    
    # create the permutation matrix
    Permutation = np.zeros([rows,columns])
    
    diag_U = np.eye(rows, dtype=complex)
    
    indices = np.nonzero(U)
    row_info = indices[0]
    col_info = indices[1]
    
    for i in range(rows):
        col = col_info[i]
        row = row_info[i]
        diag_U[col,:] = U[row,:]
        Permutation[col,row] = 1
        
    return diag_U, Permutation.T

In [ ]:
def phase(x):
    x_r = np.real(x)
    x_i = np.imag(x)
    
    raw_phi = np.arccos(x_r)
    if x_i > 0:
        t_phi = raw_phi
    else:
        t_phi = -raw_phi
        
    normalized_solution = t_phi % (2 * np.pi)
    if normalized_solution > np.pi:
        normalized_solution = normalized_solution-2*np.pi

    return normalized_solution

In [ ]:
# the all_list tell the decomposition order
# [[1,2], [2,3], [3,4]] means first use 2 eliminate 1, then use 3 eliminate 2, then use 4 eliminate 3
def U_decompose_plus(U, all_list):
    
    # pick up a machine precision
    threshold = 1e-8
    
    # get the shape of unitary
    rows, columns = U.shape
    assert(rows==columns)
    
    # create the rotation record matrix, the first row for theta, the second row for phi 
    # third row for low index, forth row for high index
    Theta = []
    Phi = []
    Low_idx = []
    High_idx = []
    
    # create the diagonal record matrix
    Diag = []
    

    for i in range(rows-1, 0, -1):
        list_i = all_list[i-1]
        row = U[i, :]
        
        for j in range(i):
            pair = list_i[j]
            
            
            # a is the number to be eliminated, b is the number to eliminate the previous one
            a = row[pair[0]]
            b = row[pair[1]]
            
            
#             if i % 3 != 0:
#                 if j == i-1:
#                     a = row[pair[1]]
#                     b = row[pair[0]]
                
                
            # choose the elimination method by position
            if pair[0] < pair[1]: 
                theta, phi = left_elimination(a,b)

                # write down the parameter
                Theta.append(theta)
                Phi.append(phi)
                Low_idx.append(pair[0])
                High_idx.append(pair[1])

                # create the rotation
                r = rotation(theta, phi, rows, pair[0], pair[1])
        

                # update Unitary
                U = np.dot(U, r)
                U= np.where(np.abs(U) < threshold, 0, U)

            else:
                theta, phi = right_elimination(b,a)

                # write down the parameter
                Theta.append(theta)
                Phi.append(phi)
                Low_idx.append(pair[1])
                High_idx.append(pair[0])

                # create the rotation
                r = rotation(theta, phi, rows, pair[1], pair[0])

                # update Unitary
                U = np.dot(U, r)
                U= np.where(np.abs(U) < threshold, 0, U)

            row = U[i, :]
            
#             if (np.count_nonzero(row) == 1):
            if j == i-1:
                entry = U[i, pair[1]]
                U[:, pair[1]] = 0
                U[i, :] = 0
                U[i, pair[1]] = entry
                flag = 0
    
    # do the permutation, and record
    U, Permutation = permutation(U)
    
    # find the phase shift
    for i in range(rows):
        x = U[i,i]/np.abs(U[i,i])
        phi_x = phase(x)
        Diag.append(phi_x)
        U[i,i] = U[i,i]*(np.cos(phi_x)-np.sin(phi_x)*1j)
    
    
    return U, Theta, Phi, Low_idx, High_idx, Diag, Permutation

Step3: Gate drop

In [ ]:
# Step1 probability
def calculate_probability_sequence(angles, threshold, N):
    abs_angles = np.abs(angles)
    angles1 = abs_angles/(threshold)
    angles2 = angles1**N
    total_magnitude = np.sum(angles2)
    probabilities = angles2 / total_magnitude
    return probabilities

In [ ]:
# sequence is the origianl angel sequence, probability is calculate using calculate_probability_sequence(angles), percentage is the proportion you want to preserve
def pick_entries_with_indices(sequence, probabilities, proportion):
    
    N = len(sequence)
    num_entries = np.floor(N*proportion).astype(int)
    
    idx_sequence = np.arange(N).astype(int)
    
    if np.count_nonzero(probabilities) < num_entries:
        num_entries = np.count_nonzero(probabilities)
    
    picked_indices = np.random.choice(idx_sequence, size=num_entries, replace=False, p=probabilities)
    picked_entries = [sequence[index] for index in picked_indices]

    # Sort the picked entries and indices based on the indices
    picked_entries, picked_indices = zip(*sorted(zip(picked_entries, picked_indices), key=lambda x: x[1]))
    
    modified_sequence = [entry if index in picked_indices else 0 for index, entry in enumerate(sequence)]

    return modified_sequence

Step4: Matric Reconstruction

In [ ]:
def make_entries_zero(Array, threshold):
    array = Array.copy()
    for i in range(len(array)):
        if np.abs(array[i]) < threshold:
            array[i] = 0
    return array

In [ ]:
# give rotation matrix with idx_1 < idx_2
def reconstruct_rotation(theta, phi, n, idx_1, idx_2):
    I = np.eye(n, dtype=complex)
    I[idx_1,idx_1] = np.cos(theta)*(np.cos(phi)+np.sin(phi)*1j)
    I[idx_1,idx_2] = -np.sin(theta)
    I[idx_2,idx_1] = np.sin(theta)*(np.cos(phi)+np.sin(phi)*1j)
    I[idx_2,idx_2] = np.cos(theta)
    
    return I

In [ ]:
def matrix_reconstruct(Theta, Phi, Low_idx, High_idx, Diag, Permutation, N):
    
    M = len(Theta)
    
    per_theta = (len([x for x in Theta if np.abs(x) == 0]) / len(Theta)) * 100
#     print("theta reduce")
#     print(per_theta)
    
    per_phi = (len([x for x in Phi if np.abs(x) == 0]) / len(Phi)) * 100
#     print("phi reduce")
#     print(per_phi)

#     new_theta = Theta
#     new_phi = Phi
    
    V = np.eye(N, dtype=complex)
    
    # rotation
    for i in range(M):
        r = reconstruct_rotation(Theta[i], Phi[i], N, Low_idx[i], High_idx[i])
        V = np.dot(r, V)
        
    # phase shift
    for i in range(N):
        V[i,:] = V[i,:]*(np.cos(Diag[i])+np.sin(Diag[i])*1j)
        
    # permutation
    V = np.dot(Permutation, V)
    
    return V

Accuracy

In [ ]:
# function to calculate the approximation accuracy
def accuracy(U,U_app):
    I = np.dot(U,np.conjugate(U_app).transpose())
    N, N = I.shape
    acc = np.trace(I)/N
    return acc

In [ ]:
def unitary_accuracy(M, U1, dec_list, maps, len_list, n, proportion1, th, N):
    
    N, N = U1.shape
    
    # mapping
    if maps:
        U1_map, col_per_matrix, row_per_matrix = mapping(U1, len_list, n)
    else:
        U1_map = U1
        
    # get the decomposition information of U1
    _, Theta1, Phi1, Low_idx1, High_idx1, Diag1, Permutation1 = U_decompose_plus(U1_map, dec_list)
    
    # modify Theta
    Theta_prob1 = calculate_probability_sequence(Theta1, th, N)
    
    k = 0
    acc1 = []
    while k < M:

        # get the new Theta from the probability distribution
        new_Theta1 = pick_entries_with_indices(Theta1, Theta_prob1, proportion1)
        new_Phi1 = Phi1
        
        # reconstruction of U2 using new angel
        U1_map_app = matrix_reconstruct(new_Theta1, new_Phi1, Low_idx1, High_idx1, Diag1, Permutation1, N)
        # approximation accuracy
        acc1_t = accuracy(U1_map,U1_map_app)
        acc1.append(acc1_t)
        
        k = k+1
        
    acc1_mean = np.mean(acc1)
    print("mean accuracy for U1:", acc1_mean)    

Reduce counting

In [ ]:
def count_numbers_less_than(numbers, threshold):
    count = 0
    for number in numbers:
        if np.abs(number) < threshold:
            count += 1
    return count

Convert sample to nodes

In [ ]:
# convert sample to nodes
def convert_binary_to_nodes(binary_array):
    nodes_array = []

    for i in range(len(binary_array)):
        if binary_array[i] == 1:
            nodes_array.append(i)

    return nodes_array

Step5: Sampling dense subgraph

In [ ]:
# define the sampleing function
# G is the graph
# G_dense_dic contains the information that tells the densest value of k nodes, k from 1 to N
# M is the total sampling numbers
# A is the adjacency matrix
# dec_list: a list give the decomposition order  
# mapping: "on" or "off", coreesponding to 1 or 0
# len_list: a list for mapping
# n: a compare number for mapping
# all_modes: (q[0], ..., q[N-1])
# proportion: 0~1 for U2
# loss is the photon number loss
# qdrift_kind: 0: defualt, 1:cut
def graph_sampling_plus(G, G_dense_dic, M, S, U, dec_list, maps, len_list, n, proportion, th, N, loss, kind):
    
#     # takagi decomposition to get Squeezing and Unitary
#     S, U = sf.decompositions.takagi(A)
    
    N, N = U.shape
    
    S_max = np.max(S)
    S = S/(S_max+1.8)
    S = np.arctanh(S)
    
    # mapping
    if maps:
        U_map, col_per_matrix, row_per_matrix = mapping(U, len_list, n)
    else:
        U_map = U
        col_per_matrix = np.eye(N)
        row_per_matrix = np.eye(N)
    
    # get the decomposition information of U
    _, Theta, Phi, Low_idx, High_idx, Diag, Permutation = U_decompose_plus(U_map, dec_list)
    
    # modify Theta
    Theta_prob = calculate_probability_sequence(Theta, th, N)
    
    
    
    k = 0
    sample = []
    acc = []
    confusion_list = []
    clique = 0
    while k < M:
#         print(k)
#         if k % 1000 == 0:
#             print(k)
        
        # get the new Theta from the probability distribution
        new_Theta = pick_entries_with_indices(Theta, Theta_prob, proportion)
#         new_Theta = Theta
        new_Phi = Phi
        
        # reconstruction of U using new angel
        U_map_app = matrix_reconstruct(new_Theta, new_Phi, Low_idx, High_idx, Diag, Permutation, N)
        # approximation accuracy
        acc_t = accuracy(U_map,U_map_app)
        acc.append(acc_t)
        
        
        # construct the circuit
        prog = sf.Program(N)
        eng = sf.Engine('gaussian')
        with prog.context as q:
            
            # r squeezing
            for i, s in enumerate(S):
                ops.Sgate(s) | q[i]
            
            # mapping's column transformation
            ops.Interferometer(col_per_matrix) | (q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8], q[9], q[10], q[11], q[12], q[13], q[14], q[15], q[16], q[17], q[18], q[19], q[20], q[21], q[22], q[23])
            
            
            # interferometer U
            for i in range(len(new_Theta)):
                ops.Rgate(new_Phi[i])       | q[Low_idx[i]]
                ops.BSgate(new_Theta[i], 0) | (q[Low_idx[i]], q[High_idx[i]])
                if np.abs(new_Theta[i]) > 0:
                    ops.LossChannel(loss) | q[Low_idx[i]]
                    ops.LossChannel(loss) | q[High_idx[i]]
                
            for i in range(N):
                ops.Rgate(Diag[i])       | q[i]
                
                
            # mapping's row transformation
            ops.Interferometer(row_per_matrix) | (q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7], q[8], q[9], q[10], q[11], q[12], q[13], q[14], q[15], q[16], q[17], q[18], q[19], q[20], q[21], q[22], q[23])
            
            
            # measurement
            ops.MeasureThreshold() | q
            

        
        results = eng.run(prog, shots=1)
        sample_t = results.samples
        sample.append(sample_t[0])
#         print(sample_t)
        
        # change the sample to the right form
        sample_sub_t = convert_binary_to_nodes(sample_t[0])
        sample_sub_t = [str(num) for num in sample_sub_t]
#         print(sample_sub_t)
        
        # calculate is this sample_sub's desity  
        if (len(sample_sub_t)>=10):
            subgraph = G.subgraph(sample_sub_t)
            density = calculate_density(subgraph)
#             density = subgraph.search([sample_sub_t], G, len(sample_sub_t), len(sample_sub_t), max_count=1)[len(sample_sub_t)][0][0]
#             print(density)
        else: 
            density = 0
        
        # find the subgraph size
        size_t = len(sample_sub_t)
#         print(size_t)

        # find out if it is the densest subgraph in this size
        if density >= G_dense_dic[size_t]-1e-5:
            confusion_list.append(1)
            if density == 1:
#                 print(kind, size_t)
                clique = clique + 1
#             print(density)
#             print(size_t)
        else: 
            confusion_list.append(0)
        
        k = k+1
        
    # calculate the frequency that successfully find the dense sub graph
    
    acc_mean = np.mean(acc)
#     print("mean accuracy for U:", acc_mean)
    
    metric = clique/M
#     print("successfully find dense subgraph:", metric)
#     print(clique)

    # manipulate the samples
    zero = np.zeros(N)
    for i in range(len(sample)):
        sample[i] = np.concatenate((sample[i], zero))
    sample_list = []
    for a in sample:
        b = list(a)
        c = [int(num) for num in b]
        sample_list.append(c)
    
        
    return sample_list, metric

Create benchmark for max clique

In [ ]:
def generate_random_graph(nodes, probability):
    G = nx.Graph()
    G.add_nodes_from(nodes)

    for i in nodes:
        for j in nodes:
            if i != j and random.random() < probability:
                G.add_edge(i, j)

    return G

In [ ]:
def calculate_density(subgraph):
    num_nodes = len(subgraph.nodes())
    num_edges = len(subgraph.edges())
    return 2 * num_edges / (num_nodes * (num_nodes - 1))

In [ ]:
def find_highest_density_subgraph(G):
    
    highest_density = 0.0
    highest_density_subgraph = None
    N = len(G)
    
    density_iter = [1.0, 1.0]
    for k_nodes in range(2, N+1):  # Iterate over all possible subgraph sizes
#         print(k_nodes)
        subgraphs = combinations(G.nodes(), k_nodes)  # Generate all combinations of nodes
        highest_density = 0
        
#         highest_density = 0.5
        
        for nodes in subgraphs:
            subgraph = G.subgraph(nodes)  # Create a subgraph from the selected nodes
            density = calculate_density(subgraph)
            if density > highest_density:
                highest_density = density
                
        print(highest_density)
        density_iter.append(highest_density)
        
#     density_iter.append(1.0)

    return density_iter

In [ ]:
# give a dictionary that contains the information that the densest density for k nodes
def create_density_dictionary(density_iter):
    density_dict = {}
    for position, density in enumerate(density_iter):
        density_dict[position] = density
    return density_dict

In [ ]:
# Generate G
# G = generate_random_graph(range(24), 0.875)

In [ ]:
# Write the graph as an adjacency list
# nx.write_adjlist(G, "new_graph.adjlist")

# Read the graph from the adjacency list
G = nx.read_adjlist("graph.adjlist")

In [ ]:
pos = nx.spring_layout(G)  # Layout algorithm for node positioning
nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', font_weight='bold')

plt.title("Graph")
plt.show()

In [ ]:
# Obtain the adjacency matrix of the graph G
A = nx.to_numpy_array(G)

In [ ]:
S, U = sf.decompositions.takagi(A)

In [ ]:
S, U = sf.decompositions.takagi(A)
S_max = np.max(S)
S = S/(S_max+0.55)

In [ ]:
n = 0
for s in S:
    n = n+s**2/(1-s**2)
print(n)

In [ ]:
S, U = sf.decompositions.takagi(A)

In [ ]:
calculate_density(G)

Calculate the information of this graph

In [ ]:
# function that find the densest subgraph in graph G of all size k
# density_iter = find_highest_density_subgraph(G)

In [ ]:
density_iter = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99523, 0.99134, 0.98814, 0.98188]

In [ ]:
density_dict = create_density_dictionary(density_iter)

In [ ]:
print(density_dict)

dec_list1: 1D

In [ ]:
list_order = [[0,1], [1,2], [2,3], [3,4], [4,5], [5,6], [6,7], [7,8], [8,9], [9,10], [10,11], [11,12], [12,13], [13,14], [14,15], [15,16], [16,17], [17,18], [18,19], [19,20], [20,21], [21,22], [22,23]]
dec_list1 = [list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order, list_order]

len_list1: 1D

In [ ]:
len_list1 = [24]

dec_list2: 2D

In [ ]:
list_order23 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6], [20,6], [21,6], [6,7], [22,7], [7,23]]
list_order22 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6], [20,6], [21,6], [6,7], [7,22]]
list_order21 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6], [20,6], [21,6], [6,7]]
list_order20 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6], [20,6], [6,21]]
list_order19 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6], [6,20]]
list_order18 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [19,5], [5,6]]
list_order17 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [18,5], [5,19]]
list_order16 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5], [5,18]]
list_order15 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [17,4], [4,5]]
list_order14 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [16,4], [4,17]]
list_order13 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4], [4,16]]
list_order12 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [15,3], [3,4]]
list_order11 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [14,3], [3,15]]
list_order10 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3], [3,14]]
list_order9 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [13,2], [2,3]]
list_order8 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [12,2], [2,13]]
list_order7 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2], [2,12]]
list_order6 = [[8,0], [9,0], [0,1], [10,1], [11,1], [1,2]]
list_order5 = [[8,0], [9,0], [0,1], [10,1], [1,11]]
list_order4 = [[8,0], [9,0], [0,1], [1,10]]
list_order3 = [[8,0], [9,0], [0,1]]
list_order2 = [[8,0], [0,9]]
list_order1 = [[0,8]]


dec_list2 = [list_order1, list_order2, list_order3, list_order4, list_order5, list_order6, list_order7, list_order8, list_order9, list_order10, list_order11, list_order12, list_order13, list_order14, list_order15, list_order16, list_order17, list_order18, list_order19, list_order20, list_order21, list_order22, list_order23]

len_list2: 2D

In [ ]:
len_list2 = [8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

Base Sampling

In [ ]:
start_time = time.time()
sample_list_base, acc_base = graph_sampling_plus(G, density_dict, 20000, S, U, dec_list1, 0, len_list1, 12, 1, 1, 1, 1, 'base:')
end_time = time.time()
execution_time_base = end_time - start_time

In [ ]:
# create path for sample_base
path_sample_base = 'sample_base.csv'
# Open the file in write mode and create a CSV writer
with open(path_sample_base, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a single row in the CSV file
    writer.writerow(sample_list_base)

Sampling

In [ ]:
acc_000 = []
acc_001 = []
acc_101 = []
acc_111 = []
M = 10000

In [ ]:
time1 = []
time2 = []
time3 = []
time4 = []

In [ ]:
sample_000 = []
sample_001 = []
sample_101 = []
sample_111 = []

In [ ]:
loss_range = [0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9]

In [ ]:
# setting: e1--000, e2--001, e3--101, e4--111
# the proportion of gate we use are 1 for e1, 0.943 for e2, 0.801 for e3, and 0.736 for e4
# then matrix approximation accuracy is 0.9998 for e2, 0.9999 for e3 and e4 (verify at the end of notebook)

for loss in loss_range:
    
    # 000
    start_time = time.time()
    sample_list1, acc1 = graph_sampling_plus(G, density_dict, M, S, U, dec_list1, 0, len_list1, 12, 1, 1, 1, loss, '000:')
    end_time = time.time()
    execution_time1 = end_time - start_time
    time1.append(execution_time1)
    sample_000.append(sample_list1)
    acc_000.append(acc1)
    
    # 001
    start_time = time.time()
    sample_list2, acc2 = graph_sampling_plus(G, density_dict, M, S, U, dec_list1, 0, len_list1, 12, 0.9711, 0.03, 100, loss, '001:')
#     sample_list2, acc2 = graph_sampling_plus(G, density_dict, M, S, U, dec_list1, 0, len_list1, 12, 0.946, 0.2, 100, loss, '001:')
    end_time = time.time()
    execution_time2 = end_time - start_time
    time2.append(execution_time2)
    sample_001.append(sample_list2)
    acc_001.append(acc2)
    
    # 101
    start_time = time.time()
    sample_list3, acc3 = graph_sampling_plus(G, density_dict, M, S, U, dec_list2, 0, len_list2, 12, 0.801, 0.025, 100, loss, '101:')
    end_time = time.time()
    execution_time3 = end_time - start_time
    time3.append(execution_time3)
    sample_101.append(sample_list3)
    acc_101.append(acc3)
    
    # 111
    start_time = time.time()
    sample_list4, acc4 = graph_sampling_plus(G, density_dict, M, S, U, dec_list2, 1, len_list2, 7, 0.712, 0.03, 100, loss, '111:')
    end_time = time.time()
    execution_time4 = end_time - start_time
    time4.append(execution_time4)
    sample_111.append(sample_list4)
    acc_111.append(acc4)

In [ ]:
# create path for sample_000
path_sample_000 = 'sample_000.csv'
# Open the file in write mode and create a CSV writer
with open(path_sample_000, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a single row in the CSV file
    writer.writerow(sample_000)
    
# create path for sample_001
path_sample_001 = 'sample_001.csv'
# Open the file in write mode and create a CSV writer
with open(path_sample_001, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a single row in the CSV file
    writer.writerow(sample_001)
    
# create path for sample_101
path_sample_101 = 'sample_101.csv'
# Open the file in write mode and create a CSV writer
with open(path_sample_101, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a single row in the CSV file
    writer.writerow(sample_101)
    
# create path for sample_111
path_sample_111 = 'sample_111.csv'
# Open the file in write mode and create a CSV writer
with open(path_sample_111, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list as a single row in the CSV file
    writer.writerow(sample_111)

Calculate the JSD, picture 6 in max clique, Fig. 11.

In [ ]:
def jensen_shannon_divergence(p, q):
    """
    Calculate the Jensen-Shannon Divergence (JSD) between two probability distributions.

    Parameters:
        p (numpy array): Probability distribution P as a 1D numpy array.
        q (numpy array): Probability distribution Q as a 1D numpy array.

    Returns:
        float: The Jensen-Shannon Divergence (JSD) between P and Q in nats.
    """
    # Ensure that the input arrays have the same length
    if len(p) != len(q):
        raise ValueError("Input arrays must have the same length.")

    # Normalize the input arrays to ensure they are valid probability distributions
    p = p / np.sum(p)
    q = q / np.sum(q)

    # Calculate the average distribution (M) as the element-wise average of P and Q
    m = 0.5 * (p + q)
    
    # Count the number of zero entries
    zero_count = np.count_nonzero(m == 0)

    # Print the result
    if zero_count>0:
        print('error: m has zero entry')


    kl_pm = 0
    for i, p_i in enumerate(p):
        if p[i] != 0:
            kl_pm = kl_pm + p[i]*np.log(p[i] / m[i])
            
    kl_qm = 0
    for i, q_i in enumerate(q):
        if q[i] != 0:
            kl_qm = kl_qm + q[i]*np.log(q[i] / m[i])
    

    # Calculate the Jensen-Shannon Divergence (JSD) in nats
    jsd = 0.5 * (kl_pm + kl_qm)

    return jsd

In [ ]:
def process_sample(sample):
    # Remove 0 from the sample
    sample_without_zeros = [element for element in sample if element != 0]
    # Sort the sample in descending order
    sorted_sample = sorted(sample_without_zeros, reverse=True)
    return sorted_sample

In [ ]:
# the element in base_sample and sample should be str
def calculate_jsd(base_sample, sample):
    
    for i, s in enumerate(base_sample):
        # Example string representing a list
        string_representation = s

        # Convert the string to a list using ast.literal_eval()
        try:
            result_list = ast.literal_eval(string_representation)
        except ValueError as e:
            # If the string is not a valid Python literal, handle the exception
            print(f"Error: {e}")
            result_list = None
            
        s = process_sample(result_list)
        
        string_list = str(s)
        
        base_sample[i] = string_list
        
    for i, s in enumerate(sample):
        # Example string representing a list
        string_representation = s

        # Convert the string to a list using ast.literal_eval()
        try:
            result_list = ast.literal_eval(string_representation)
        except ValueError as e:
            # If the string is not a valid Python literal, handle the exception
            print(f"Error: {e}")
            result_list = None
            
        s = process_sample(result_list)
        
        string_list = str(s)
        
        sample[i] = string_list
        
        
    
    # Combine the two lists and convert to a set to remove duplicates
    combined_set = set(base_sample + sample)

    # Convert the set back to a list to get a distinct list
    distinct_sample = list(combined_set)
    
    # Calculate the probability for base_sample
    probability_list1 = []
    total_elements1 = len(base_sample)
    element_count1 = Counter(base_sample)
    for element in distinct_sample:
        occurrences = element_count1[element] if element in element_count1 else 0
        probability = occurrences / total_elements1
        probability_list1.append(probability)
        
    # Calculate the probability for sample
    probability_list2 = []
    total_elements2 = len(sample)
    element_count2 = Counter(sample)
    for element in distinct_sample:
        occurrences = element_count2[element] if element in element_count2 else 0
        probability = occurrences / total_elements2
        probability_list2.append(probability) 
        
    # Calculate the Jensen-Shannon Divergence (JSD) between P and Q
    jsd_result = jensen_shannon_divergence(probability_list1, probability_list2)
#     print("Jensen-Shannon Divergence:", jsd_result)   
    
    return jsd_result

In [ ]:
# Function to calculate the mean value for a group of 3 numbers
def calculate_mean(group):
    return sum(group) / len(group)

In [ ]:
# create path for base_sample
path_sample_base = 'sample_base.csv'
# Open the file in read mode and create a CSV reader
with open(path_sample_base, 'r') as file:
    reader = csv.reader(file)

    # Read the first row from the CSV file
    row = next(reader)

    # Convert the row elements to integers (if needed)
    sample_base_read = [element for element in row] 
    
x_axis = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

In [ ]:
# create path for sample
path_sample = 'sample_000.csv'
# Open the file in read mode and create a CSV reader
with open(path_sample, 'r') as file:
    reader = csv.reader(file)

    # Read the first row from the CSV file
    row = next(reader)

    # Convert the row elements to integers (if needed)
    sample_read1 = [element for element in row]
    

# create path for sample
path_sample = 'sample_001.csv'
# Open the file in read mode and create a CSV reader
with open(path_sample, 'r') as file:
    reader = csv.reader(file)

    # Read the first row from the CSV file
    row = next(reader)

    # Convert the row elements to integers (if needed)
    sample_read2 = [element for element in row]
    
# create path for sample
path_sample = 'sample_101.csv'
# Open the file in read mode and create a CSV reader
with open(path_sample, 'r') as file:
    reader = csv.reader(file)

    # Read the first row from the CSV file
    row = next(reader)

    # Convert the row elements to integers (if needed)
    sample_read3 = [element for element in row]
    
# create path for sample
path_sample = 'sample_111.csv'
# Open the file in read mode and create a CSV reader
with open(path_sample, 'r') as file:
    reader = csv.reader(file)

    # Read the first row from the CSV file
    row = next(reader)

    # Convert the row elements to integers (if needed)
    sample_read4 = [element for element in row]
    
    

jsd1 = []

for i, sample in enumerate(sample_read1):
    
    # Your string representation of the list of lists
    string_representation = sample_read1[i]

    # Convert the string to an actual list of lists
    try:
        list_of_lists = ast.literal_eval(string_representation)
    except ValueError as e:
        # If the string_representation is not a valid Python literal, ValueError will be raised.
        print("Error:", e)
        list_of_lists = None
    
    string_list = [str(sub_list) for sub_list in list_of_lists]
    
    jsd = calculate_jsd(sample_base_read, string_list)
    
    jsd1.append(jsd)
    
#     print(jsd)
    
jsd2 = []

for i, sample in enumerate(sample_read2):
    
    # Your string representation of the list of lists
    string_representation = sample_read2[i]

    # Convert the string to an actual list of lists
    try:
        list_of_lists = ast.literal_eval(string_representation)
    except ValueError as e:
        # If the string_representation is not a valid Python literal, ValueError will be raised.
        print("Error:", e)
        list_of_lists = None
    
    string_list = [str(sub_list) for sub_list in list_of_lists]
    
    jsd = calculate_jsd(sample_base_read, string_list)
    
    jsd2.append(jsd)
    
#     print(jsd)
    
    
jsd3 = []

for i, sample in enumerate(sample_read3):
    
    # Your string representation of the list of lists
    string_representation = sample_read3[i]

    # Convert the string to an actual list of lists
    try:
        list_of_lists = ast.literal_eval(string_representation)
    except ValueError as e:
        # If the string_representation is not a valid Python literal, ValueError will be raised.
        print("Error:", e)
        list_of_lists = None
    
    string_list = [str(sub_list) for sub_list in list_of_lists]
    
    jsd = calculate_jsd(sample_base_read, string_list)
    
    jsd3.append(jsd)
    
#     print(jsd)
    
    
    
jsd4 = []

for i, sample in enumerate(sample_read4):
    
    # Your string representation of the list of lists
    string_representation = sample_read4[i]

    # Convert the string to an actual list of lists
    try:
        list_of_lists = ast.literal_eval(string_representation)
    except ValueError as e:
        # If the string_representation is not a valid Python literal, ValueError will be raised.
        print("Error:", e)
        list_of_lists = None
    
    string_list = [str(sub_list) for sub_list in list_of_lists]
    
    jsd = calculate_jsd(sample_base_read, string_list)
    
    jsd4.append(jsd)
    
    
plt.figure(figsize=(8, 4))    
    
# Plot data on each subplot
plt.plot(x_axis, jsd1, color='blue', label='baseline', marker='x', markersize=16, linewidth=1)
plt.plot(x_axis, jsd2, color='green', label='rotation-cut', marker='^', markersize=16, linewidth=1)
plt.plot(x_axis, jsd3, color='orange', label='decomposition-opt', marker='o', markersize=16, linewidth=1)
plt.plot(x_axis, jsd4, color='red', label='full-opt', marker='s', markersize=16, linewidth=1)
# plt.set_title('6by6 structure', fontsize=15)
# plt.xlabel('loss', fontsize=15)
# plt.ylabel('JSD-dense', fontsize=15)
plt.tick_params(axis='x', labelsize=28)
plt.tick_params(axis='y', labelsize=28)
# Hide the tick labels on both x and y axes
# plt.xticks([])
# plt.yticks([])
# axs[0, 0].legend(fontsize=12, loc='lower left')  # Set the fontsize for the legend labels to 12
# plt.gca().invert_yaxis()  # Invert the y-axis
# plt.legend(loc='lower left', fontsize=16)

# Save the plot as a PDF
output_filename = 'Fig 11.svg.svg'
plt.savefig(output_filename, format='svg', bbox_inches='tight')

# Show the plot
plt.show()

Calculate the approximation accuracy for the four experiment settings, this is just a verification to show our gate drop ratio is reasonable.

In [ ]:
unitary_accuracy(1000, U, dec_list1, 0, len_list1, 12, 1, 1, 100)

In [ ]:
unitary_accuracy(1000, U, dec_list1, 0, len_list1, 12, 0.9711, 0.03, 100)

In [ ]:
unitary_accuracy(1000, U, dec_list2, 0, len_list2, 12, 0.801, 0.025, 100)

In [ ]:
unitary_accuracy(1000, U, dec_list2, 1, len_list2, 7, 0.712, 0.03, 100)